In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib widget
%pdb off

from pyCascade import probePost, physics, utils, probeReadWrite
from pyCascade.probeReadWrite import read_probes_file_switch
from filloutVentilationStats import *
from matplotlib import pyplot as plt
from matplotlib import cm, colors
import matplotlib.ticker as ticker
import matplotlib.colors as mcolors
import numpy as np
import scipy as sp
import os
from IPython.core.debugger import set_trace
import pandas as pd
import seaborn as sns
from cycler import cycler
import plotly.express as px
import plotly
import plotly.graph_objects as go
from plotly.offline import plot
from plotly.subplots import make_subplots
from IPython.display import display, HTML
import statsmodels.api as sm
import warnings
import ast
from scipy.optimize import minimize
import seaborn as sns
from tqdm import tqdm

plotly.offline.init_notebook_mode()
display(HTML(
    '<script type="text/javascript" async src="https://cdnjs.cloudflare.com/ajax/libs/mathjax/2.7.1/MathJax.js?config=TeX-MML-AM_SVG"></script>'
))

plt.rcParams['figure.dpi'] = 140
im_scaling = .75
plt.rcParams['figure.figsize'] = [6.4 * im_scaling, 4.8 * im_scaling]

############ Universal ################
scratch_home = os.getenv('SCRATCH') #need to set SCRATCH (even if there is no real SCRATCH) to the location where results are written
scratch_dir = f'{scratch_home}/Cascade/city_block_cfd'
home_dir = !pwd
home_dir = home_dir[0]

display(scratch_dir)
display(home_dir)
plt.close('all')

# Setup

## Runs

In [ ]:
multiRun_dir = f"{home_dir}/CHARLES/multiRuns/"
plotFolder = f"{multiRun_dir}"

## Read in results

In [ ]:
def evalStringAsList(s):
    if isinstance(s, str) and s[0] == "[" and s[-1] == "]":
        return ast.literal_eval(s)
    else:
        return s

In [ ]:
flowStatsMI = pd.read_csv(f"{multiRun_dir}/flowStatsMI.csv", index_col = [0,1])
roomVentilationMI = pd.read_csv(f"{multiRun_dir}/roomVentilationMI.csv", index_col = [0,1])


In [ ]:
# runSubset = []
# runSubset += [
#     2530, 2531, 2532, 
#     2540, 2541, 2542, 
#     2550, 2551, 2552, 
#     2560, 2561, 2562, 
#     3210, 3211, 3212, 
#     3220, 3221, 3222, 
#     3230, 3231, 3232, 
#     3240, 3241, 3242, 
# ]

# runSubset += [
#     2460, 2461, 2462, 
#     2470, 2471, 2472, 
#     2480, 2481, 2482, 
#     2490, 2491, 2492, 
#     3160, 3161, 3162, 
#     3170, 3171, 3172, 
#     3180, 3181, 3182, 
#     3190, 3191, 3192, 
# ]

# flowStatsMI = flowStatsMI.loc[runSubset]
# roomVentilationMI = roomVentilationMI.loc[runSubset]

In [ ]:
flowStatsMI = combine_stats(flowStatsMI, ["WS", "delT", "SS", "C", "A", "slAll"], index_col = "csId")
roomVentilationMI = combine_stats(roomVentilationMI,   ["WS", "delT", "SS", "C", "A", "slAll"], index_col = "csId")

# flowStatsMI = flowStatsMI[flowStatsMI["slAll"] == True]
# roomVentilationMI = roomVentilationMI[roomVentilationMI["slAll"] == True]

In [ ]:
# 2*flowStatsMI2.loc[2495]["rms-sn_prod(abs(u))-Norm"]**2 - flowStatsMI.loc[2460]["rms-sn_prod(abs(u))-Norm"]**2 - flowStatsMI.loc[2530]["rms-sn_prod(abs(u))-Norm"]**2

In [ ]:
# replace "_sl_" with "_h_-1-0_" on the second level of both MultiIndexes
def replace_sl_with_h(lbl):
    if type(lbl) == str:
        return lbl.replace("sl", "h_-1-0")
    return lbl

def replace_sl_with_h_df(df, level=1):
    df.rename(index=replace_sl_with_h, level=level, inplace=True)
    for col in df.columns:
        if "windowKeys" in col:
            df[col] = df[col].apply(replace_sl_with_h)
        if "houseType" in col:
            df[col] = df[col].apply(lambda s: s.replace("sl", "-1-0"))
    return df


flowStatsMI = replace_sl_with_h_df(flowStatsMI)
roomVentilationMI = replace_sl_with_h_df(roomVentilationMI)

flowStatsMI.loc[flowStatsMI["houseType"] == "-1-0", "slAll"] = True
roomVentilationMI.loc[roomVentilationMI["houseType"] == "-1-0", "slAll"] = True

In [ ]:
runsNoInt = {
    257: {'A': 45, 'WS': 2, 'C': 2},
    258: {'A': 45, 'WS': 4, 'C': 2},
    259: {'A': 0,  'WS': 2, 'C': 2},
    260: {'A': 0,  'WS': 4, 'C': 2},

    325: {'A': 45, 'WS': 2, 'C': 3},
    326: {'A': 45, 'WS': 4, 'C': 3},
    327: {'A': 0,  'WS': 2, 'C': 3},
    328: {'A': 0,  'WS': 4, 'C': 3},
}

In [ ]:
pcOverwrite = False

def getNoIntRun(df, A, WS, C):
    filtered = df[(df['A'] == A) & (df['WS'] == WS) & (df['C'] == C)]
    if not filtered.empty:
        return filtered.index[0]
    return None

if pcOverwrite or os.path.exists(f"{multiRun_dir}/pointCloudStatsNoIntMI.csv") == False:
    # pointCloud pressure probes
    runIndex = flowStatsMI.index.get_level_values(0).unique()
    runIndex = np.sort(runIndex)
    pcStats = pd.DataFrame()

    def addB(name):
        if name.split("_")[-1][0] != "B":
            return f"{name}_B"
        return name

    for run in runIndex:
        A = flowStatsMI.loc[run, "A"].iloc[0]
        WS = flowStatsMI.loc[run, "WS"].iloc[0]
        C = flowStatsMI.loc[run, "C"].iloc[0]
        R = getNoIntRun(pd.DataFrame(runsNoInt).T, A, WS, C)
        V = run % 10
        if V == 0:
            stats = pd.read_csv(f"{home_dir}/CHARLES/config{C}/R{R%100}/probes/flowStats-40000to119000.csv", index_col = 0)
            stats.index = stats.index.map(addB)
            stats.columns = [str(col).replace('-119000', '') for col in stats.columns]
            stats.columns = [col + "-noInt" for col in stats.columns]
        stats["run"] = run
        stats["run-noInt"] = R * 10 + V
        stats = replace_sl_with_h_df(stats, level=0)
        pcStats = pd.concat([pcStats, stats], axis = "rows")

    pcStatsMI = pcStats.reset_index()
    pcStatsMI = pcStatsMI.set_index(['run', 'index'])
    pcStatsMI.to_csv(f"{multiRun_dir}/pointCloudStatsNoIntMI.csv")
else:
    pcStatsMI = pd.read_csv(f"{multiRun_dir}/pointCloudStatsNoIntMI.csv", index_col=[0,1])

In [ ]:
# Check that the window identifiers are the same

def check_equivalence(s1, s2, tol = 2e-1):
    s1.dropna(inplace=True)
    s2.dropna(inplace=True)
    if s1.dtype == float and s2.dtype == float:
        return (abs(s1 - s2) < tol)
    return s1 == s2

match_cols = ["x", "z", "orientation"]
match_cols = [f"{col}-noInt" for col in match_cols]
flowStatsMI[pcStatsMI.columns] = np.nan
flowStatsMI.update(pcStatsMI)

for col in flowStatsMI.columns:
    if "noInt" in col and flowStatsMI[col].dtype != float:
        match_cols.append(col)

for match_col in match_cols:
    ref_col = match_col.removesuffix("-noInt")
    s_match = check_equivalence(flowStatsMI[ref_col], flowStatsMI[match_col])
    if s_match.all():
        flowStatsMI.drop(match_col, inplace=True, axis="columns")
    else:
        display(flowStatsMI.loc[s_match == False, :])
        raise ValueError(f"Mismatch in {ref_col} and {match_col}")
            
# flowStatsMI

In [ ]:
# Extract velocity components as separate variables
u = flowStatsMI["comp(u_avg,0)-noInt"].values  # x-component
v = flowStatsMI["comp(u_avg,2)-noInt"].values  # y-component
w = flowStatsMI["comp(u_avg,1)-noInt"].values  # z-component

# Calculate magnitude
velocity_magnitude = np.sqrt(u**2 + v**2 + w**2)

# Calculate spherical angles
azimuth = np.degrees(np.arctan2(v, u))  # Angle in xy-plane
elevation = np.degrees(np.arccos(w/velocity_magnitude))  # Angle from z-axis

# Now use these variables for visualization
fig = px.scatter(x=u, y=v, color=flowStatsMI["openingType"], 
                 labels={"x": "u component", "y": "w component"}, 
                 title="Velocity Components")
fig.show()

# Create a scatter plot showing the angular distribution
fig2 = px.scatter(x=azimuth, y=elevation, color=flowStatsMI["openingType"],
                  size=velocity_magnitude,
                  labels={"x": "Azimuth Angle (°)", "y": "Elevation Angle (°)"},
                  title="Velocity Vector Orientation")
fig2.show()

In [ ]:
# +------------------------------------------------------------------------+
# |                     Mixture-of-Six-Planes via RANSAC                    |
# +------------------------------------------------------------------------+

import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

def fit_plane_ransac(V_norm, epsilon=1e-3, num_iters=1000):
    """
    RANSAC-fit a plane through the origin to normalized vectors.
    Returns best‐fit unit normal and inlier mask.
    """
    best_n = None
    best_inliers = None
    best_count = 0
    N = V_norm.shape[0]
    
    for _ in range(num_iters):
        # pick two distinct vectors at random
        i, j = np.random.choice(N, 2, replace=False)
        vi, vj = V_norm[i], V_norm[j]
        
        # candidate normal
        n = np.cross(vi, vj)
        norm_n = np.linalg.norm(n)
        if norm_n < 1e-6:
            continue
        n /= norm_n
        
        # make hemisphere choice deterministic
        if n[np.argmax(np.abs(n))] < 0:
            n = -n
        
        # count inliers
        dots = np.abs(V_norm @ n)
        inliers = dots < epsilon
        count = inliers.sum()
        
        if count > best_count:
            best_count = count
            best_n = n
            best_inliers = inliers
    
    return best_n, best_inliers

def find_planes(u, v, w, n_planes=6, epsilon=1e-3, num_iters=1000):
    """
    Sequentially RANSAC-fit up to six planes to (u,v,w) velocity data.
    Returns:
      normals      : K×3 array of plane normals (K ≤ 6)
      assignments  : length-N array of plane indices [0…K-1] or -1 for outliers
    """
    # stack and normalize
    V = np.stack((u, v, w), axis=1)
    norms = np.linalg.norm(V, axis=1, keepdims=True)
    V_norm = np.zeros_like(V)
    nonzero = norms[:,0] > 0
    V_norm[nonzero] = V[nonzero] / norms[nonzero]
    
    N = V_norm.shape[0]
    assignments = np.full(N, -1, dtype=int)
    normals = []
    available = np.ones(N, dtype=bool)
    
    for k in range(n_planes):
        idxs = np.where(available)[0]
        if idxs.size < 2:
            break
        
        sub_V = V_norm[available]
        n, inliers_sub = fit_plane_ransac(sub_V, epsilon, num_iters)
        if n is None:
            break
        
        normals.append(n)
        
        # map back to full mask
        in_full = np.zeros(N, dtype=bool)
        in_full[idxs[inliers_sub]] = True
        
        assignments[in_full] = k
        available[in_full] = False
    
    return np.array(normals), assignments

def plot_planes_and_vectors(normals, u, v, w, assignments):
    """
    Plot the velocity directions colored by assigned plane and fitted plane normals on a unit sphere.

    Parameters
    ----------
    normals : array_like, shape (K,3)
        Unit normals of fitted planes.
    u, v, w : array_like, shape (N,)
        Velocity components.
    assignments : array_like, shape (N,)
        Plane indices [0..K-1] for each vector, or -1 for outliers.
    """
    import numpy as np
    import matplotlib.pyplot as plt

    # Stack & normalize velocities
    V = np.stack((u, v, w), axis=1)
    norms_V = np.linalg.norm(V, axis=1, keepdims=True)
    V_norm = np.zeros_like(V)
    nz = norms_V[:, 0] > 0
    V_norm[nz] = V[nz] / norms_V[nz]

    # Normalize plane normals
    normals_norm = normals / np.linalg.norm(normals, axis=1, keepdims=True)
    K = normals_norm.shape[0]

    # Create unit-sphere mesh
    phi = np.linspace(0, np.pi, 40)
    theta = np.linspace(0, 2 * np.pi, 40)
    phi, theta = np.meshgrid(phi, theta)
    X = np.sin(phi) * np.cos(theta)
    Y = np.sin(phi) * np.sin(theta)
    Z = np.cos(phi)

    # Prepare colormap
    cmap = plt.get_cmap('tab10', K)

    # Plot
    fig = plt.figure(figsize=(8, 8))
    ax = fig.add_subplot(111, projection='3d')
    ax.plot_wireframe(X, Y, Z, rstride=4, cstride=4, linewidth=0.5,
                     color='gray', alpha=0.5)

    # Scatter velocities by assignment
    for k in range(K):
        mask = (assignments == k)
        if np.any(mask):
            ax.scatter(
                V_norm[mask, 0], V_norm[mask, 1], V_norm[mask, 2],
                marker='o', s=5, color=cmap(k), label=f'velocities: plane {k}'
            )
    # Plot outliers if any
    out_mask = (assignments == -1)
    if np.any(out_mask):
        ax.scatter(
            V_norm[out_mask, 0], V_norm[out_mask, 1], V_norm[out_mask, 2],
            marker='x', s=30, color='k', label='outliers'
        )

    # Plot normals colored to match planes
    for k, n in enumerate(normals_norm):
        ax.scatter(
            n[0], n[1], n[2], marker='^', s=100,
            color=cmap(k), edgecolor='k', label=f'normal {k}'
        )

    ax.set_xlabel('X')
    ax.set_ylabel('Y')
    ax.set_zlabel('Z')
    ax.set_title('Velocity Directions and Plane Normals on Unit Sphere')
    ax.set_box_aspect([1, 1, 1])
    ax.legend(loc='upper right', bbox_to_anchor=(1.3, 1))
    plt.tight_layout()
    plt.show()

# === USAGE ===

# 1) Load or define your velocity components here:
#    (each must be a 1D NumPy array of the same length)
# u = np.array([...])
# v = np.array([...])
# w = np.array([...])

# For demo, we’ll simulate random data below—comment out in real use:
# np.random.seed(0)
# u, v, w = (np.random.randn(300), np.random.randn(300), np.random.randn(300))

# 2) Fit up to six planes:
epsilon = 1e-1       # tolerance for inliers: adjust to your noise level
num_iters = 2000     # RANSAC trials per plane
magnitude = np.sqrt(u**2 + v**2 + w**2)
mag_lim = 0.1
valid = magnitude > mag_lim
u_valid = u[valid]
v_valid = v[valid]
w_valid = w[valid]

print(f"Valid data points: {valid.sum()} out of {len(valid)} based on magnitude > {mag_lim}")
normals, assignments = find_planes(u_valid, v_valid, w_valid, 8, epsilon, num_iters)

print(f"Fitted {len(normals)} plane normals:")
print(normals)

n_per_plane = [(assignments==k).sum() for k in range(len(normals))]
print("Counts per plane:", n_per_plane)
print("Outliers:", (assignments==-1).sum())

# 3) Visualize:
plot_planes_and_vectors(normals, u_valid, v_valid, w_valid, assignments)

In [ ]:
# Compute azimuth and elevation in degrees
# Azimuth: angle in the XY plane from +X axis
# Elevation: angle from the XY plane up to the vector
azimuth   = np.degrees(np.arctan2(normals[:, 1], normals[:, 0]))
elevation = np.degrees(np.arcsin(normals[:, 2]))

# Plot
plt.figure(figsize=(6, 6))
plt.scatter(azimuth, elevation, marker='x', s=100)

# Annotate each point with its normal index
for i, (a, e) in enumerate(zip(azimuth, elevation)):
    plt.text(a, e, f'{i}', fontsize=12,
             ha='right', va='bottom')

plt.xlabel('Azimuth (°)')
plt.ylabel('Elevation (°)')
plt.title('Plane Normals: Azimuth vs Elevation')
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
px.scatter(data_frame=flowStatsMI, x="EP_p_avg-noInt", y='EP_p_avg', symbol = "houseType", color = "roomType")

In [ ]:
g = 10
beta = 0.0034
rho = 1.225
hm = 6
window_dim = hm/2/4
A = window_dim ** 2

def getWindBuoyantP(rho, flowParams):
    p_w = flowParams["p_w"]
    z = flowParams["z"]
    delT = flowParams["delT"]
    delrho = -rho * beta * delT
    return (delrho * g * z) + p_w # delP is outdoor minus indoor, while p0/rho is indoor minus outdoor, driving positive flow into the room (oppiste textbook)

def flowFromP(rho, C_d, A, delp):
    delp=np.array(delp)
    S = np.sign(delp)
    return S * C_d * A * np.sqrt(2 * abs(delp) / rho)

def CFromFlow(rho, q, A, delp):
    delp = np.array(delp, dtype=float)
    # prepare output filled with NaNs
    C = np.full_like(delp, np.nan, dtype=float)
    # mask non‐NaN, non‐zero delp
    mask = ~np.isnan(delp) & (delp != 0)
    S = np.sign(delp[mask])
    C[mask] = q[mask] / (S * A[mask] * np.sqrt(2 * np.abs(delp[mask]) / rho))
    return C

def flowField(p_0, rho, flowParams):
    C_d = flowParams["C_d"]
    A = flowParams["A"]
    rooms = flowParams["rooms"]
    delP = -np.matmul(rooms, p_0) + getWindBuoyantP(rho, flowParams) 
    return flowFromP(rho, C_d, A, delP)

def getC(p_0, rho, flowParams):
    A = flowParams["A"]
    q = flowParams["q"]
    rooms = flowParams["rooms"]
    delP = -np.matmul(rooms, p_0) + getWindBuoyantP(rho, flowParams)
    return CFromFlow(rho, q, A, delP)

def qObjective(p_0, rho, flowParams):
    qs = flowField(p_0, rho, flowParams)
    rooms = flowParams["rooms"]
    qRooms = np.matmul(rooms.T, qs)
    return np.sum(qRooms**2)

def findOptimalP0(rho, flowParams):
    bounds = np.array([np.min(getWindBuoyantP(rho, flowParams)), np.max(getWindBuoyantP(rho, flowParams))])
    x0 = np.mean(bounds)
    NRooms = flowParams["rooms"].shape[1]
    bounds = np.tile(bounds, (NRooms, 1))
    x0 = np.tile(x0, NRooms)
    return minimize(qObjective, x0=x0, bounds=bounds, args=(rho, flowParams))

In [ ]:
def matchObjective(x, rho, flowParams, weight):
    """
    Objective = 
      1) sum of squared errors between predicted opening flows and target flows
      2) + weight × variance(C_d)
    x = [p0_1 ... p0_N, Cd_1 ... Cd_M]
    """
    rooms   = flowParams["rooms"]
    N       = rooms.shape[1]    # # rooms
    M       = rooms.shape[0]    # # openings

    # unpack decision vector
    p0 = x[:N]
    Cd = x[N:]

    # compute driving pressures
    params    = flowParams.copy()
    params["C_d"] = Cd
    delP      = -rooms.dot(p0) + getWindBuoyantP(rho, params)

    # 1) predicted opening flows
    qs_pred = flowFromP(rho, Cd, params["A"], delP)

    # 2) flow-matching error (per opening)
    q_target = params["q"]
    f1 = np.sum((qs_pred - q_target)**2)

    # 3) uniform-C penalty (variance)
    meanCd = np.mean(Cd)
    f2     = np.sum((Cd - meanCd)**2)

    return f1 + weight * f2

def findOptimalP0AndC(rho, flowParams, weight=1, disp=False):
    """
    Minimize matchObjective over p0 (N vars) and Cd (M vars).
    Returns OptimizeResult with .x = [p0*, Cd*].
    """
    rooms = flowParams["rooms"]
    N     = rooms.shape[1]
    M     = rooms.shape[0]

    # bounds for p0: between min/max wind-buoyancy pressures
    WBP      = getWindBuoyantP(rho, flowParams)
    p_bounds = [(np.min(WBP), np.max(WBP))] * N

    # bounds for Cd: e.g. [1e-3, 5.0]
    C_bounds = [(1e-3, 5.0)] * M
    bounds   = p_bounds + C_bounds

    # initial guess: mid-range p0, mean(C_d)
    x0 = np.concatenate([
        np.full(N, np.mean(WBP)),
        np.full(M, np.mean(flowParams["C_d"]))
    ])

    res = minimize(
        matchObjective,
        x0=x0,
        args=(rho, flowParams, weight),
        bounds=bounds,
        method="L-BFGS-B",
        options={"disp": disp}
    )
    return res


In [ ]:
flowParams = {
    "C_d": [1],
    "A": [1],
    "p_w": [0],
    "z": [3],
    "delT": [3],
    "q": [1],
    "rooms": [[1]]
}

flowParams = utils.dict_apply(np.array)(flowParams)

p_0s = np.array([0])

display(flowField(p_0s, rho, flowParams))
display(qObjective(p_0s, rho, flowParams))
# combine p0s and C_d into the input vector for jointObjective
x = np.concatenate((p_0s, flowParams["C_d"]))
display(matchObjective(x, rho, flowParams, 1e2))
display(getC(p_0s, rho, flowParams))

In [ ]:
flowParamsSet = {
    "C_d": [1, 1, 1, 1],
    "A": [1, 1, 1, 1],
    "p_w": [1, 3, 1, -3],
    "z": [3, 3, 6, 3],
    "delT": [-3, 0, 3, 0],
    "q": [1, 2, -1, -2],
    "rooms": [[1], [1], [1], [1]]
}

flowParams = utils.dict_apply(np.array)(flowParamsSet)

p_0s = np.array([1])

display(flowField(p_0s, rho, flowParams))
display(qObjective(p_0s, rho, flowParams))
x = np.concatenate((p_0s, flowParams["C_d"]))
display(matchObjective(x, rho, flowParams, 1e2))
display(getC(p_0s, rho, flowParams))

In [ ]:
plt.figure()
p0s = np.linspace(min(getWindBuoyantP(rho, flowParams)), max(getWindBuoyantP(rho, flowParams)), 100)
qs = [qObjective(np.array([p0]), rho, flowParams) for p0 in p0s]
plt.plot(p0s, qs)
plt.xlabel("p0")
plt.ylabel("$\sum q^2$")


In [ ]:
optResults = findOptimalP0(rho, flowParams)
p_0s = optResults.x

display(optResults)
display(flowField(p_0s, rho, flowParams))
display(qObjective(p_0s, rho, flowParams))
display(getC(p_0s, rho, flowParams))

In [ ]:
flowParams = utils.dict_apply(np.array)(flowParamsSet)
p_0s = np.array([1])

optResults = findOptimalP0AndC(rho, flowParams, weight=1e-1, disp=True)
n_rooms = flowParams["rooms"].shape[1] 
p_0s = optResults.x[0:n_rooms]
flowParams["C_d"] = optResults.x[n_rooms:]

display(optResults)
display(flowField(p_0s, rho, flowParams))
display(getC(p_0s, rho, flowParams))

In [ ]:
flowParamsSet = {
    "C_d": [1, 1, 1, 1, 10],
    "A": [1, 1, 1, 1, 3],
    "p_w": [1, 3, 1, -3, 0],
    "z": [3, 3, 6, 3, 0],
    "delT": [-3, 0, 3, 0, 0],
    "q": [2, 2, -1, -3, -4],
    "rooms": [[1, 0], [1, 0], [0, 1], [0, 1], [1, -1]]
}

flowParams = utils.dict_apply(np.array)(flowParamsSet)
p_0s = np.array([1, -1])

display(flowField(p_0s, rho, flowParams))
display(qObjective(p_0s, rho, flowParams))
display(getC(p_0s, rho, flowParams))

In [ ]:
optResults = findOptimalP0(rho, flowParams)
p_0s = optResults.x

display(optResults)
display(flowField(p_0s, rho, flowParams))
display(qObjective(p_0s, rho, flowParams))
display(getC(p_0s, rho, flowParams))


In [ ]:
flowParams = utils.dict_apply(np.array)(flowParamsSet)
p_0s = np.array([1, -1])

optResults = findOptimalP0AndC(rho, flowParams, weight=1e-1)
n_rooms = flowParams["rooms"].shape[1]
p_0s = optResults.x[0:n_rooms]
flowParams["C_d"] = optResults.x[n_rooms:]

display(optResults)
display(flowField(p_0s, rho, flowParams))
display(getC(p_0s, rho, flowParams))

In [ ]:
from scipy.optimize import minimize

def getC_ds_AofA(aCorner, aCross, aDual, aSingle, aSinA, aCosA, AofA = 180, roomType="corner"):
    AofA *= np.pi / 180
    C_d = aSinA * np.sin(AofA) + aCosA * np.cos(AofA)
    if roomType == "corner":
        return C_d + aCorner
    if roomType == "cross":
        return C_d + aCross
    if roomType == "dual":
        return C_d + aDual
    if roomType == "single":
        return C_d + aSingle
    
def getC_ds_AofA2(aCorner, aCross, aSingle, aCornerSinA, aCrossSinA, AofA = 180, roomType="corner"):
    AofA *= np.pi / 180
    if roomType == "corner":
        return aCornerSinA * np.sin(AofA) + aCorner
    if roomType == "cross":
        return aCrossSinA * np.sin(AofA) + aCross
    if roomType == "dual":
        return aCornerSinA * np.sin(AofA) + aCorner
    if roomType == "single":
        return aSingle

def getC_ds_AofA3(aCorner, aCross, aDual, aSingle, aSinA, aCosA, sSin2a, aCos2A, AofA = 180, roomType="corner"):
    AofA *= np.pi / 180
    C_d = aSinA * np.sin(AofA) + aCosA * np.cos(AofA) + sSin2a * np.sin(2 * AofA) + aCos2A * np.cos(2 * AofA)
    if roomType == "corner":
        return C_d + aCorner
    if roomType == "cross":
        return C_d + aCross
    if roomType == "dual":
        return C_d + aDual
    if roomType == "single":
        return C_d + aSingle

def getC_ds_EP(aCorner, aCross, aDual, aSingle, aShear, aNormal, shear=0, normal=0, roomType="corner"):
    C_d = aShear * shear + aNormal * normal
    if roomType == "corner":
        return C_d + aCorner
    if roomType == "cross":
        return C_d + aCross
    if roomType == "dual":
        return C_d + aDual
    if roomType == "single":
        return C_d + aSingle

def getC_ds_All(aCorner, aCross, aDual, aSingle, aSinA, aCosA,  aShear, aNormal, aWS, AofA = 180, shear=0, normal=0, WS=0, roomType="corner"):
    AofA *= np.pi / 180
    C_d = aSinA * np.sin(AofA) + aCosA * np.cos(AofA) + aShear * shear + aNormal * normal + aWS * WS
    if roomType == "corner":
        return C_d + aCorner
    if roomType == "cross":
        return C_d + aCross
    if roomType == "dual":
        return C_d + aDual
    if roomType == "single":
        return C_d + aSingle

def getC_ds_opening(aX, aZ, aXSinA, aZSinA, aXCosA, aZCosA, AofA=0, openingType="xwindow"):
    AofA *= np.pi / 180
    if openingType == "xwindow":
        return aX + aXSinA * np.sin(AofA) + aXCosA * np.cos(AofA)
    if openingType == "zwindow":
        return aZ + aZSinA * np.sin(AofA) + aZCosA * np.cos(AofA)

def getC_ds(params, typeC_d="AofA", AofA=0, shear=0, normal=0, openingType="xwindow", roomType="corner"):
    if typeC_d == "AofA":
        return getC_ds_AofA(*params, AofA=AofA, roomType=roomType)
    if typeC_d == "AofA2":
        return getC_ds_AofA2(*params, AofA=AofA, roomType=roomType)
    if typeC_d == "AofA3":
        return getC_ds_AofA3(*params, AofA=AofA, roomType=roomType)
    if typeC_d == "EP":
        return getC_ds_EP(*params, shear=shear, normal=normal, roomType=roomType)
    if typeC_d == "All":
        return getC_ds_All(*params, AofA=AofA, shear=shear, normal=normal, WS=1, roomType=roomType)
    if typeC_d == "opening":
        return getC_ds_opening(*params, openingType=openingType)



def update_flow_and_ventilation(flowStatsMI, roomVentilationMI, paramsC_d, useDoors=True, pTypes = {"p-noInt": "p_avg-noInt"}, typeC_d="AofA"):
    flowStatsMI["cosAofA"] = np.round(np.cos(flowStatsMI["AofA"] * np.pi / 180), 2)
    flowStatsMI["sinAofA"] = np.round(np.sin(flowStatsMI["AofA"] * np.pi / 180), 2)

    for pType in pTypes:
        roomVentilationMI[f"{pType}-success"] = False
    print(f"Processung {roomVentilationMI.shape[0]} rooms")
    for (run, room), row in tqdm(roomVentilationMI.iterrows()):
        flowParams = {
            "C_d": [],
            "A": [],
            "z": [],
            "delT": [],
            "q": [],
            "rooms": [],
        }
        windowKeyCols = roomVentilationMI.columns[
            roomVentilationMI.columns.str.contains("windowKeys")
        ].tolist()
        windowKeys = row[windowKeyCols].dropna()

        for pType in pTypes:
            flowParams[pType] = []
        for windowKey in windowKeys:
            for pType, pCol in pTypes.items():
                flowParams[pType].append(flowStatsMI.loc[(run, windowKey), pCol])
            C_d = getC_ds(
                paramsC_d,
                typeC_d = typeC_d,
                AofA=flowStatsMI.loc[(run, windowKey),"AofA"], 
                shear=flowStatsMI.loc[(run, windowKey),"EP_shear"], 
                normal=flowStatsMI.loc[(run, windowKey),"EP_normal"],
                openingType = flowStatsMI.loc[(run, windowKey),"openingType"],
                roomType = row["roomType"],
                )
            flowParams["C_d"].append(C_d)
            flowParams["A"].append(A)
            flowParams["z"].append(flowStatsMI.loc[(run, windowKey), "y"])  # y is vertical in simulation
            flowParams["delT"].append(row["mean-T-room"])
            flowParams["q"].append(flowStatsMI.loc[(run, windowKey), "mean-mass_flux"])
            if "dual" in room and useDoors:
                roomCord = windowKey.split("_")[1]
                if roomCord == "0-1":
                    roomRow = [1, 0]
                elif roomCord == "1-1":
                    roomRow = [0, 1]
                else:
                    raise Exception(f"Unrecognized room {roomCord} in dual room")
            else:
                roomRow = [1]
            flowParams["rooms"].append(roomRow)

        if "dual" in room and useDoors:
            H = 3
            for pType in pTypes:
                flowParams[pType].append(0)
            flowParams["C_d"].append(1)
            flowParams["A"].append(A * 3)
            flowParams["z"].append(H / 2)
            flowParams["delT"].append(row["mean-T-room"])
            qRooms = np.matmul(np.array(flowParams["rooms"]).T, np.array(flowParams["q"]))
            flowParams["q"].append(np.diff(qRooms)[0])
            flowParams["rooms"].append([1, -1])

        flowParams = utils.dict_apply(np.array)(flowParams)

        sinAofAs = []
        cosAofAs = []
        for i, windowKey in enumerate(windowKeys):
            sinAofAs.append(flowStatsMI.loc[(run, windowKey), "sinAofA"])
            cosAofAs.append(flowStatsMI.loc[(run, windowKey), "cosAofA"])

        roomVentilationMI.loc[(run, room), "sinAofA"] = np.mean(sinAofAs)
        roomVentilationMI.loc[(run, room), "cosAofA"] = np.mean(cosAofAs)

        for pType in pTypes:
            NRooms = flowParams["rooms"].shape[1]
            flowParams["p_w"] = flowParams[pType]
            p0_meas = [row["mean-p-room"] for i in range(NRooms)]
            C_ds = getC(np.array(p0_meas), rho, flowParams)

            for i, windowKey in enumerate(windowKeys):
                flowStatsMI.loc[(run, windowKey), f"{pType}-C_d"] = C_ds[i]

            n_rooms = flowParams["rooms"].shape[1]
            optResults = {"optp0": findOptimalP0(rho, flowParams),
                            "optp0Cd": findOptimalP0AndC(rho, flowParams, weight=1e-1)}
            for optType, optResult in optResults.items():
                flowParamsOpt = flowParams.copy()
                p0 = optResult.x[0:n_rooms]
                if optType == "optp0Cd":
                    flowParamsOpt["C_d"] = optResult.x[n_rooms:]

                roomVentilationMI.loc[(run, room), f"{pType}_{optType}-p0"] = np.mean(p0)
                roomVentilationMI.loc[(run, room), f"{pType}_{optType}-success"] = optResult.success
                roomVentilationMI.loc[(run, room), f"{pType}_{optType}-fun"] = optResult.fun
                qs = flowField(np.array(p0), rho, flowParamsOpt)

                for i, windowKey in enumerate(windowKeys):
                    flowStatsMI.loc[(run, windowKey), f"{pType}_{optType}-q_model"] = qs[i]
                    flowStatsMI.loc[(run, windowKey), f"{pType}_{optType}-netq_model"] = abs(qs[i])
                    flowStatsMI.loc[(run, windowKey), f"{pType}_{optType}-C_d"] = flowParamsOpt["C_d"][i]

                if "dual" in room and useDoors:
                    qs = qs[0:-1]
                roomVentilationMI.loc[(run, room), f"{pType}_{optType}-q_model"] = np.sum(abs(np.array(qs))) / 2
        
            

    return flowStatsMI, roomVentilationMI


In [ ]:
flowStatsMI.loc[flowStatsMI["openingType"] == "skylight", "AofA"] = 180

# [1.16, 1.31, 1.15, 1, 0, 0]

flowStatsMI, roomVentilationMI = update_flow_and_ventilation(flowStatsMI, roomVentilationMI, [1, 1, 1, 1, 0, 0], useDoors=True,
    pTypes = {
        "pW": "mean-sn_prod(p)",
        "pEP": "EP_p_avg",
        "pEP-noInt": "EP_p_avg-noInt",
        "p-noInt": "p_avg-noInt"
    }
)


In [ ]:
flowStatsMI["u_mag-noInt"] = np.sqrt(flowStatsMI["comp(u_avg,0)-noInt"]**2 + flowStatsMI["comp(u_avg,1)-noInt"]**2 + flowStatsMI["comp(u_avg,2)-noInt"]**2)

model_cols = [col for col in flowStatsMI.columns if "noInt" in col]
model_cols = [col for col in model_cols if "x-" not in col]
model_cols = [col for col in model_cols if "y-" not in col]
model_cols = [col for col in model_cols if "z-" not in col]
model_cols = [col for col in model_cols if "run" not in col]
model_cols = [col for col in model_cols if "q-" not in col]
model_cols = [col for col in model_cols if "comp(u_avg,0)" not in col]
model_cols = [col for col in model_cols if "comp(u_avg,2)" not in col]
model_cols += ["roomType", "openingType", "AofA", "sinAofA", "cosAofA", "slAll"]
model_cols += ["mean-mass_flux", "mean-sn_prod(abs(u))-Norm"]
model_data = flowStatsMI[model_cols]
model_data.to_csv(f"{multiRun_dir}/intEmulation.csv")

model_data

In [ ]:
# flowStatsMINoSL = flowStatsMI[flowStatsMI["AofA"] % 1 == 0].copy()
# roomVentilationMINoSL = roomVentilationMI[roomVentilationMI["sinAofA"].notna()].copy()

In [ ]:
pVar = "p-noInt_optp0Cd-q_model"
dfFit = flowStatsMI.dropna(subset=[pVar])
# Reshape x into a 2D column vector
x = dfFit[pVar].values.reshape(-1, 1)  # Ensure it's 2D
y = dfFit["mean-mass_flux"].values  # Keep y as 1D

# Solve for the slope (forcing intercept = 0)
CfitWindow, _, _, _ = np.linalg.lstsq(x, y, rcond=None)
CfitWindow = CfitWindow[0]

# Print the fitted coefficient
print("Fitted Coefficient:", CfitWindow)

# Fit C for each room type separately
room_types = dfFit['roomType'].unique()
CfitRoomTypes = {}

for room_type in room_types:
    dfRoomType = dfFit[dfFit['roomType'] == room_type]
    x = dfRoomType[pVar].values.reshape(-1, 1)  # Ensure it's 2D
    y = dfRoomType["mean-mass_flux"].values  # Keep y as 1D

    # Solve for the slope (forcing intercept = 0)
    Cfit, _, _, _ = np.linalg.lstsq(x, y, rcond=None)
    CfitRoomTypes[room_type] = Cfit[0]

# Print the fitted coefficients for each room type
for room_type, Cfit in CfitRoomTypes.items():
    print(f"Fitted Coefficient for {room_type}: {Cfit}")



In [ ]:
# Create the scatter plot
plt.figure(figsize=(10, 6))
sns.scatterplot(data=flowStatsMI, x=pVar, y="mean-mass_flux", hue="roomType", style="openingType")

# Overlay the lines from the fitted C values
x_vals = np.linspace(flowStatsMI["mean-mass_flux"].min(), flowStatsMI["mean-mass_flux"].max(), 100)

# Overall fitted line
y_vals_overall = x_vals * CfitWindow
plt.plot(x_vals, y_vals_overall, label='Overall Fit', color='black')

# Fitted lines for each room type
for room_type, Cfit in CfitRoomTypes.items():
    y_vals_room = Cfit * x_vals
    plt.plot(x_vals, y_vals_room, label=f'{room_type} Fit')

plt.legend()
plt.xlabel('Mean Mass Flux')
plt.ylabel(pVar)
plt.title('Scatter Plot with Fitted Lines')

In [ ]:
"""
Angles:
array([ 1.17340176,  1.32334274,  1.15833887,  0.00804852, -0.03391678,
        0.01388756])
0.14092224654844634

Shear/Normal:
array([ 1.16847614e+00,  1.31839869e+00,  1.17020841e+00,  1.62123154e-04,
        1.76884283e-04, -4.20435224e-04])
0.14068568319124428

Openning:
array([ 1.17383905e+00,  1.25949764e+00,  1.04268126e-05,  3.76369301e-05,
       -4.24760102e-05,  1.37200509e-04])
0.14497885547335324

AofA2:
array([ 1.17132613e+00,  1.31910337e+00,  2.03823868e-04, -4.39502373e-04,
        2.03392091e-04])
0.14071399814356256
"""

In [ ]:
# def qRMSE(df):
#     return np.sqrt(np.mean((df["p-noInt-q_model"] - df["mean-mass_flux"])**2))

# def objective(params, flowStatsMI = flowStatsMINoSL, roomVentilationMI = roomVentilationMINoSL):
#     display(params)
#     flowStatsMI, roomVentilationMI = update_flow_and_ventilation(flowStatsMI, roomVentilationMI, params, typeC_d="AofA3")
#     RMSE = qRMSE(roomVentilationMI)
#     display(RMSE)
#     return RMSE
# initial_guess = [ 1.17340176,  1.32334274,  1.15833887,  0.00804852, -0.03391678, 0.01388756, 0, 0]
# result = minimize(objective, initial_guess, method='Nelder-Mead')

In [ ]:
plotdf = roomVentilationMI.copy()
plotdf["x"] = plotdf["mean-p-room"]**0.5
plotdf["y"] = plotdf["p-noInt_optp0Cd-p0"]**0.5
px.scatter(data_frame=plotdf, x="x", y='y', symbol = "roomType", color = "delT")

In [ ]:
px.box(data_frame=flowStatsMI.sort_values(["AofA", "roomType"]), x="p-noInt-C_d", color = "AofA", facet_col = "roomType")

In [ ]:
px.box(data_frame=flowStatsMI.sort_values(["AofA", "roomType"]), x="p-noInt_optp0Cd-C_d", color = "AofA", facet_col = "roomType")

In [ ]:
px.box(data_frame=flowStatsMI.sort_values(["AofA", "roomType"]), x="p-noInt_optp0Cd-C_d", facet_col = "openingType")

In [ ]:
from sklearn.cluster import KMeans
from sklearn.decomposition import TruncatedSVD

# Filter and sort the data
flowStatsMINoSL = flowStatsMI[flowStatsMI["AofA"] % 1 == 0]
flowStatsMINoSL.sort_values(by=["WS", "AofA"], inplace=True)

# Define substrings to keep and drop
subStrKeep = ["p-noInt-C_d", "sinAofA", "cosAofA", "roomType", "WS", "EP_normal", "EP_shear", "mean-mass_flux", "openingType"]
meanKmeansDf = flowStatsMINoSL[subStrKeep]

# Convert roomType to numeric
meanKmeansDf["roomType"] = meanKmeansDf["roomType"].astype('category').cat.codes
meanKmeansDf["openingType"] = meanKmeansDf["openingType"].astype('category').cat.codes

# Perform z-score normalization
mean = np.mean(meanKmeansDf, axis=0)
std = np.std(meanKmeansDf, axis=0)
meanKmeansDf = (meanKmeansDf - mean) / std

# Apply K-Means Algorithm
k = 4
kmeans = KMeans(n_clusters=k, random_state=0, n_init='auto')
kmeans.fit(meanKmeansDf)

# Assign cluster labels
flowStatsMINoSL["labels"] = kmeans.labels_.astype(str)

# Perform SVD decomposition
svd = TruncatedSVD(n_components=3)
meanKmeansDfLowDim = svd.fit_transform(meanKmeansDf)

# Visualize the clusters
fig = px.scatter_3d(data_frame=flowStatsMINoSL, x=meanKmeansDfLowDim[:, 0], y=meanKmeansDfLowDim[:, 1], z=meanKmeansDf["p-noInt-C_d"], color="labels")
fig.update_traces(marker=dict(size=3))
fig.update_layout(width=1200, height=800, legend=dict(x=0, y=1.1))
fig.show()

In [ ]:
# Perform SVD decomposition
svd = TruncatedSVD(n_components=meanKmeansDf.shape[1])
meanKmeansDfLowDim = svd.fit_transform(meanKmeansDf)

# Plot the magnitude of the eigenvalues
plt.figure()
plt.plot(range(1, len(svd.explained_variance_) + 1), svd.explained_variance_, marker='o')
plt.xlabel('Component Number')
plt.ylabel('Eigenvalue Magnitude')
plt.title('Magnitude of Eigenvalues')
plt.grid(True)
plt.show()

In [ ]:
corrDf = meanKmeansDf.copy()
corrDf['roomType'] = flowStatsMINoSL['roomType']

# Split the data by roomType
room_types = corrDf['roomType'].unique()

# Plot correlation matrices for each roomType
fig, axes = plt.subplots(2, 2, figsize=(10, 8))
axes = axes.flatten()

for ax, room_type in zip(axes, room_types):
    room_data = corrDf[corrDf['roomType'] == room_type]
    room_data = room_data.drop(columns='roomType')
    corr_matrix = room_data.corr()
    sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', fmt='.1f', linewidths=0.5, ax=ax)
    ax.set_title(f'Correlation Matrix for Room Type {room_type}')

plt.tight_layout()

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

# Define the features (independent variables) and target (dependent variable)
features = ['AofA', 'roomType', 'WS', 'EP_normal', 'EP_shear', 'mean-mass_flux']
X = meanKmeansDf[features]
y = meanKmeansDf['p-noInt-C_d']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create and fit the linear regression model
model = LinearRegression()
model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = model.predict(X_test)

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f'Mean Squared Error: {mse}')
print(f'R^2 Score: {r2}')

# Display the coefficients
coefficients = pd.DataFrame(model.coef_, features, columns=['Coefficient'])
print(coefficients)

In [ ]:
px.scatter(data_frame=flowStatsMI, x="mean-mass_flux", y='p-noInt-q_model', color = "AofA_resid")

In [ ]:
import seaborn as sns

# Create the scatter plot
plt.figure(figsize=(10, 6))
sns.scatterplot(data=flowStatsMI, x="mean-mass_flux", y='pEP-q_model', hue="roomType", style="roomType")

# Overlay the lines from the fitted C values
x_vals = np.linspace(flowStatsMI["mean-mass_flux"].min(), flowStatsMI["mean-mass_flux"].max(), 100)

# Overall fitted line
y_vals_overall = CfitWindow * x_vals
plt.plot(x_vals, y_vals_overall, label='Overall Fit', color='black')

# Fitted lines for each room type
for room_type, Cfit in CfitRoomTypes.items():
    y_vals_room = Cfit * x_vals
    plt.plot(x_vals, y_vals_room, label=f'{room_type} Fit')

plt.legend()
plt.xlabel('Mean Mass Flux')
plt.ylabel('pEP-q_model')
plt.title('Scatter Plot with Fitted Lines')
plt.show()

In [ ]:
px.scatter(data_frame=roomVentilationMI, x="mean-mass_flux", y='p-noInt-q_model', symbol = "houseType", color = "roomType")

In [ ]:
px.scatter(data_frame=roomVentilationMI, x="mean-mass_flux", y='pEP-q_model', symbol = "houseType", color = "roomType")

In [ ]:
roomVentilationMI["sinAofA"]

In [ ]:
px.scatter(data_frame=roomVentilationMI, x="mean-mass_flux", y='pEP-q_model', symbol = "houseType", color = "sinAofA")

In [ ]:
dfFit = roomVentilationMI.dropna(subset=["p-noInt-q_model"])
# Reshape x into a 2D column vector
x = dfFit["mean-mass_flux"].values.reshape(-1, 1)  # Ensure it's 2D
y = dfFit["p-noInt-q_model"].values  # Keep y as 1D

# Solve for the slope (forcing intercept = 0)
CfitRooms, _, _, _ = np.linalg.lstsq(x, y, rcond=None)
CfitRooms = CfitRooms[0]

# Print the fitted coefficient
print("Fitted Coefficient:", CfitRooms)

In [ ]:
CfitRooms = 1
roomVentilationMI["q_model-diff"] = roomVentilationMI["p-noInt-q_model"]/CfitRooms - roomVentilationMI["mean-mass_flux"]
roomVentilationMI["q_model-error"] = roomVentilationMI["q_model-diff"] / roomVentilationMI["mean-mass_flux"]

fig, axs = plt.subplots(2, 2, constrained_layout=True, figsize=(8, 6))
conditions = [(2, 0), (2, 45), (3, 0), (3, 45)]
for ax, (C, A) in zip(axs.flat, conditions):
    plotdf = roomVentilationMI[(roomVentilationMI["C"] == C) & (roomVentilationMI["A"] == A)]
    im = ax.scatter(plotdf['x'], plotdf['z'], s=8*im_scaling, c=plotdf["q_model-error"], cmap='PiYG', edgecolors="black", linewidths=.1, vmin=-2, vmax=2)
fig.colorbar(im, ax=axs, orientation='vertical', fraction=0.02, pad=0.04)

fig, axs = plt.subplots(2, 2, constrained_layout=True, figsize=(8, 6))
for ax, (C, A) in zip(axs.flat, conditions):
    plotdf = roomVentilationMI[(roomVentilationMI["C"] == C) & (roomVentilationMI["A"] == A)]
    im = ax.scatter(plotdf['x'], plotdf['z'], s=8*im_scaling, c=plotdf["q_model-diff"], cmap='RdBu', edgecolors="black", linewidths=.1, vmin=-.25, vmax=.25)
fig.colorbar(im, ax=axs, orientation='vertical', fraction=0.02, pad=0.04)

In [ ]:
flowStatsMI["q_model-diff"] = flowStatsMI["p-noInt-q_model"]/CfitRooms - flowStatsMI["mean-mass_flux"]
flowStatsMI["q_model-error"] = flowStatsMI["q_model-diff"] / flowStatsMI["mean-mass_flux"]

fig, axs = plt.subplots(2, 2, constrained_layout=True, figsize=(8, 6))
conditions = [(2, 0), (2, 45), (3, 0), (3, 45)]
for ax, (C, A) in zip(axs.flat, conditions):
    plotdf = flowStatsMI[(flowStatsMI["C"] == C) & (flowStatsMI["A"] == A)]
    im = ax.scatter(plotdf['x'], plotdf['z'], s=8*im_scaling, c=plotdf["q_model-error"], cmap='PiYG', edgecolors="black", linewidths=.1, vmin=-2, vmax=2)
fig.colorbar(im, ax=axs, orientation='vertical', fraction=0.02, pad=0.04)

fig, axs = plt.subplots(2, 2, constrained_layout=True, figsize=(8, 6))
conditions = [(2, 0), (2, 45), (3, 0), (3, 45)]
for ax, (C, A) in zip(axs.flat, conditions):
    plotdf = flowStatsMI[(flowStatsMI["C"] == C) & (flowStatsMI["A"] == A)]
    im = ax.scatter(plotdf['x'], plotdf['z'], s=8*im_scaling, c=plotdf["q_model-diff"], cmap='RdBu', edgecolors="black", linewidths=.1, vmin=-.25, vmax=.25)
fig.colorbar(im, ax=axs, orientation='vertical', fraction=0.02, pad=0.04)